In [19]:
import pandas as pd

In [20]:
df = pd.read_csv('./Dataset/data_bersih_label_v2.csv')
df

,tweet_clean,label
0,"['ken', 'tau', 'ranking', 'pasca', 'thomas', '...",P
1,"['bravo', 'india', 'winners', 'thomas', 'cup',...",NT
2,"['lee', 'chong', 'wei', 'ga', 'juara', 'thomas...",P
3,"['udh', 'gk', 'benernya', 'mata', 'kyknya', 'u...",N
4,"['iya', 'zelin', 'resiana', 'ex', 'wd', 'juara...",NT
...,...,...
442,"['indonesia', 'menang', 'thomas', 'cup', 'sea'...",P
443,"['india', 'has', 'beaten', 'indonesia', 'to', ...",NT
444,"['tuan', 'nyonya', 'shesar', 'hiren', 'rhustav...",P
445,"['nnton', 'thomas', 'cup', 'nnton', 'nya', 'ma...",NT


In [21]:
x = df['tweet_clean']
y = df['label']

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2))

x = vectorizer.fit_transform(x)

print(x)

  (0, 4294)	0.2001929610617684
  (0, 723)	0.2415882272938789
  (0, 30)	0.2415882272938789
  (0, 101)	0.2415882272938789
  (0, 885)	0.2415882272938789
  (0, 5619)	0.11603211048967946
  (0, 5376)	0.13483389559806264
  (0, 4120)	0.2001929610617684
  (0, 4422)	0.2415882272938789
  (0, 5184)	0.2415882272938789
  (0, 2781)	0.2415882272938789
  (0, 4290)	0.1710594791213803
  (0, 722)	0.2415882272938789
  (0, 29)	0.2415882272938789
  (0, 100)	0.2415882272938789
  (0, 881)	0.03776379536872808
  (0, 5617)	0.11159409339400951
  (0, 5360)	0.03776379536872808
  (0, 4119)	0.2001929610617684
  (0, 4415)	0.35470801878963826
  (0, 5182)	0.20706276350361838
  (0, 2779)	0.22631046618461878
  (1, 1229)	0.26677731462903165
  (1, 3306)	0.25399930286508315
  (1, 790)	0.26677731462903165
  :	:
  (445, 3886)	0.6573727902500854
  (445, 3329)	0.2817136996408516
  (445, 3932)	0.21897811944673132
  (445, 5362)	0.05593298769494319
  (445, 881)	0.0513785208204204
  (445, 5360)	0.0513785208204204
  (446, 2410)	0.2731

In [23]:
from imblearn.over_sampling import SMOTE

x_resampled, y_resampled = SMOTE().fit_resample(x, y)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=0)

In [25]:
print(X_train.shape)
print(X_test.shape)

(537, 6026)
(135, 6026)


In [26]:
from sklearn import svm

clf = svm.SVC(kernel='linear')

In [27]:
clf.fit(X_train, y_train)
predict = clf.predict(X_test)

In [28]:
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, accuracy_score

In [29]:
print(accuracy_score(y_test, predict))

0.8222222222222222


In [30]:
print(confusion_matrix(y_test, predict))

[[43  1  1]
 [ 0 34 11]
 [ 2  9 34]]


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

def evaluasi(model, x_train, y_train, x_test, y_test) :
    prediksi_train = model.predict(x_train)
    prediksi_test = model.predict(x_test)
    print('Akurasi data latih = {}'.format(accuracy_score(y_train, prediksi_train)))
    print('Akurasi presisi data latih = {}'.format(precision_score(y_train, prediksi_train, average='macro')))
    print('Akurasi recall data latih = {}'.format(recall_score(y_train, prediksi_train, average='macro')))
    print("================================================")
    print('Akurasi data uji = {}'.format(accuracy_score(y_test, prediksi_test)))
    print('Akurasi presisi data uji = {}'.format(precision_score(y_test, prediksi_test, average='macro')))
    print('Akurasi recall data uji = {}'.format(recall_score(y_test, prediksi_test, average='macro')))

In [32]:
def show_best_hyperparameter(model, hyperparameters):
    for key, value in hyperparameters.items() :
        print('Best '+key+':', model.get_params()[key])

In [34]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

hyperparameters = dict(
    C = [0.001, 0.01, 0.1, 1.0],
    kernel = ['linear', 'poly', 'rbf', 'sigmoid'],
    degree = [1, 2, 3, 4, 5],
    gamma = ['scale', 'auto'],
    coef0 = [0.0, 0.01, 0.02, 0.1, 0.2],
    decision_function_shape = ['ovo', 'ovr'],
    probability = [True, False],
    break_ties = [True, False]
)

clf = RandomizedSearchCV(SVC(), hyperparameters, random_state=42, cv=5, scoring='accuracy')

clf.fit(X_train, y_train)

evaluasi(clf, X_train, y_train, X_test, y_test)
print('===============================')
# show_best_hyperparameter(clf, hyperparameters)

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\USER\

Akurasi data latih = 0.994413407821229
Akurasi presisi data latih = 0.9944236371202663
Akurasi recall data latih = 0.994413407821229
Akurasi data uji = 0.8222222222222222
Akurasi presisi data uji = 0.8224710876884789
Akurasi recall data uji = 0.8222222222222223
